# Algoritam grananja sa ograničavanjem

Algoritam grananja sa ograničavanjem (eng. *branch and bound*, skraćeno *BnB*) se najčešće koristi za nalaženje optimalnog rešenja problema kombinatorne optimizacije gde je broj potencijalnih rešenja eksponencijalan u odnosu na veličinu ulaza. Sa algoritmom ćemo se najbolje upoznati kroz njegovu primenu na problem ranca i problem $n$ dama.

## Problem ranca

Neka je dat ranac veličine $w$ i $n$ predmeta veličine $v_i$ i cene $c_i$, $1 \le i \le n$. Potrebno je odabrati koji predmeti će biti smešteni u ranac tako da je njihova ukupna cena što veća. Predmeti se mogu smestiti u ranac ukoliko njihova ukupna veličina nije veća od veličine ranca. Ovaj problem je u literaturi poznat kao *0/1 knapsack problem*.

Na prvi pogled deluje da se ovaj problem može rešiti **pohlepnim algoritmom** gde se predmeti biraju u opadajućem poretku prema vrednostima odnosa njihove cene i težine, odnosno prema ceni po jedinici težine. Međutim, vrlo lako se može generisati kontraprimer. Neka imamo ranac veličine 8 i tri predmeta, od kojih je jedan veličine 7 i cene $21$, a preostala dva veličine 4 i cene $11$. Odnos cena/veličina je najveći kod prvog predmeta i iznosi $21/7=3$, a kod druga dva predmeta je manji i iznosi $11/4<3$. Prema pohlepnom algoritmu bira se prvi predmet i rešenje je 21, a optimalno rešenje ima vrednost 22 i podrazumeva izbor druga dva predmeta.

U literaturi se ovaj problem često razmatra kao uvodni problem **dinamičkog programiranja**. Ideja ovog algoritma je da se redom optimalno ispunjavaju ranci veličina $1, 2, 3, \dots$ sve dok se ne napuni ranac veličine $w$. Induktivnom hipotezom se pretpostavlja da su optimalno napunjeni ranci svih veličina do $w$. Kada se razmatra ranac veličine $w$, važi da je 

$$s_w = \max_i (s_{w-v_i} + c_i),$$

gde se maksimum računa po svim predmetima koji trenutno nisu popunjeni u rancu manje veličine. Vrednost $s_x$ predstavlja ukupnu cenu u rancu veličine $x$. Drugim rečima, rancima manjih veličina pokušavamo da dodamo još jedan predmet, tako da se maksimizuje ukupna cena sa dodatim predmetom. Ovaj algoritam efikasno rešava slučajeve kada su veličine ranca i predmeta celi brojevi. Međutim, dinamičko programiranje ne nalazi uvek optimalno rešenje kada veličine nisu celi brojevi.

Problem ranca je NP-težak i za njegovo rešavanje u opštem slučaju potreban je algoritam eksponencijalne složenosti. Primer takvog algoritma je **gruba sila** kojom se može generisati svih $2^n$ permutacija i među njima odabrati ona koja je optimalna. Primer takvog rešavanja je dat na donjoj slici. Slika predstavlja ranac veličine 10 i pet predmeta, *A, B, C, D* i *E*, datih svojim veličinama i cenama. Na slici je prikazan tzv. postupak grananja, gde svaka leva grana drveta označava da predmet ulazi u rešenje, a desna da ne ulazi. Na taj način se u listovima drveta mogu izgenerisati sva moguća rešenja. U primeru se ispostavlja da je vrednost optimalnog rešenja 235. 

<img src="assets/knapsack1.jpg" width="700px">

Gruba sila se donekle može popraviti korišćenjem **bektrekinga**. Nije potrebno razmatrati sve kombinacije: ukoliko u nekom trenutku, dodavajući trenutni predmet, dobijemo rešenje koje nije dopustivo, odnosno rešenje gde je ukupna veličina predmeta veća od veličine ranca, celo podstablo na čelu sa čvorom koji predstavlja taj predmet možemo da ignorišemo. Na slici su takva podstabla prikazana crvenom bojom. U primeru koji razmatramo imamo veoma mali broj predmeta, pa bektreking ne dobija toliko na značaju. U praksi, on može dosta ubrzati rešenje.

<img src="assets/knapsack2.jpg" width="700px">

Bektreking se dodatno može popraviti korišćenjem **BnB** algoritma. Ideja je da za svaki čvor pamtimo najbolje rešenje podstabla, odnosno koliko uključivanje najbolje solucije u podstablu poboljšava rešenje. Ukoliko je trenutno najbolje rešenje bolje od opcije da se u nekom podstablu izabere najbolje rešenje, čitavo stablo se može zanemariti. Na slici su takva podstabla označena crnom bojom. U primeru, podstabla sa korenom _E_ vrede najviše 30, podstabla sa korenom <i>D</i> vrede najviše $95 + 30 = 125$, podstabla sa korenom *C* vrede najviše $100 + 95 + 30 = 225$, itd.

<img src="assets/knapsack3.jpg" width="700px">

### Implementacija BnB algoritma za problem ranca

Iskoristićemo `queue` paket kojim se implementira red sa FIFO strukturom. Koristićemo funkcije `get` i `put` reda kojim se vrednosti uzimaju i stavljaju u red. 

In [1]:
import queue

Klasa `Item` će predstavljati jedan predmet opisan svojom težinom i vrednošću.

In [2]:
class Item:
    def __init__(self, weight, value):
        self.weight = weight
        self.value = value

Klasa `Node` će predstavljati jedan čvor u BnB drvetu. Svojstvo `level` će predstavljati nivo drveta. Broj nivoa odgovara broju predmeta.  Svojstvo `weight` predstavlja ukupnu težinu predmeta krećući se od korena. Svojstvo `value` predstavlja ukupnu vrednost predmeta krećući se od korena. Svojstvo `bound` predstavlja najveću vrednost za podstablo od trenutnog čvora. 

In [3]:
class Node:
    def __init__(self, level, weight, value):
        self.level = level
        self.weight = weight
        self.value = value
        self.bound = 0.0

Funkcija `bound` izračunava najveću vrednost za podstablo sa korenom `u`. Vrednost izračunavamo krećući se niz podstablo sve dok ukupna veličina ne prelazi veličinu ranca.

In [4]:
def bound(u, knapsack_weight, items):
    
    if (u.weight >= knapsack_weight):
        return 0
    
    total_value = u.value 
    l = u.level + 1
    total_weight = u.weight
    
    while l < len(items) and total_weight + items[l].weight <= knapsack_weight:
        total_weight += items[l].weight
        total_value += items[l].value
        l += 1
        
    return total_value

Funkcija `knapsack` izračunava najveću vrednost korišćenjem BnB algoritma. 

In [5]:
def knapsack(knapsack_weight, items):
    
    # Q ce predstavljati red u koji cemo smestati cvorove koji se trenutno posecuju
    Q = queue.Queue()
   
    # cvor za inicijalizaciju reda sa nivoom -1 i tezinom i vrednoscu nul
    u = Node(-1, 0, 0)
    
    # inicijalizujemo red
    Q.put(u)
    max_value = 0
    
    while (not Q.empty()):
        
        # u je trenutni cvor koji se razmatra
        # pod predpostavkom da ce se on zadrzati izracunavaju se dalje vrednosti
        u = Q.get()
        
        # ako smo na nivou lista, razmatramo dalje
        if (u.level == len(items) - 1): 
            continue
            
        # v ovde predstavlja cvor za nivo nize od cvora u 
        v = Node(u.level + 1, u.weight + items[u.level + 1].weight, u.value + items[u.level + 1].value)
 
        # provera da li se povecava maksimalna vrednost uzimajuci cvor v u obzir 
        if v.weight <= knapsack_weight and v.value > max_value:
            max_value = v.value
            
        # v ce se dalje razmatrati ukoliko je najbolja vrednost veca od trenutne najbolje
        v.bound = bound(v, knapsack_weight, items)
        if (v.bound > max_value):
            Q.put(v)
            
        # v ovde predstavlja cvor za nivo nize od u pod pretpostavkom da nije uzet cvor v
        v = Node(u.level + 1, u.weight, u.value)
        v.bound = bound(v, knapsack_weight, items) 
        if (v.bound > max_value):
            Q.put(v)    
            
    return max_value

Testiraćemo kako funkcija rad: 

In [6]:
knapsack_weight = 10.0
items = [Item(2, 40), Item(3.14, 50), Item(1.98, 100), Item(5, 95), Item(3, 30)]

In [7]:
print('Maksimalna vrednost:', knapsack(knapsack_weight, items))

Maksimalna vrednost: 235


## Problem *n* dama

Problem $n$ dama se sastoji u raspoređivanju $n$ dama na šahovsku tablu dimenzija $n \times n$ tako da nikoje dve dame ne napadaju jedna drugu. Dve dame napadaju jedna drugu ukoliko se nalaze u istom redu ili koloni, ili na istoj dijagonali.

Kod **bektreking** algoritma se dame postavljaju na šahovsku tablu redom po kolonama nadesno, počev od najlevlje. Kada se nova dama postavi na neko polje u trenutnoj koloni, razmatra se da li postoji bilo koje međusobno napadanje sa svakom od dama u prethodnim kolonama. Ukoliko ne postoji, dama na tom polju se dodaje u rešenje, a ukoliko postoji, vrši se vraćanje jedan korak unazad. 

Razmotrimo kako se ovaj algoritam može ubrzati primenom **BnB**. Pretpostavimo da smo, na primer, na šahovskoj tabli $8 \times 8$ postavili prvu damu na polje $(4, 1)$. Sva polja koja ona napada su označena zelenom bojom na donjoj slici.

<img src="assets/queen1.jpg" width="300px">

Kada razmatramo gde treba da postavimo drugu damu, ne moramo razmatrati svih 8 mogućnosti. Dovoljno je uočiti sva polja koja napada prva dama u toj koloni, pa njih i ne razmatrati. Ovo sužava broj potencijalnih mogućnosti na 5. Postavimo na primer drugu damu na polje $(6, 2)$. Na donjoj slici su sada označena sva polja koja napadaju obe dame zelenom boju. Za treću damu je, umesto 8, preostalo samo 3 mogućnosti, što značajno sužava prostor rešenja.

<img src="assets/queen2.jpg" width="300px">

Ideja je dakle da se efikasno odredi koja polja su u trenutnoj koloni slobodna za postavljanje nove dame, tako što će se efikasnije ažurirati koja polja su napadnuta. Dama se uvek dodaje u novu kolonu, tako da nikad nema bojazni da će se dve dame nalaziti u istoj koloni. Pri dodavanju svake nove dame, jednostavno se može označiti ceo red u koji se ona dodaje kao napadnut. Kada se nova dama dodaje, proverava se da li je trenutni red zauzet. Ostalo je još da se vidi kako efikasno proveriti da li se neka dama već nalazi u istoj koloni.

Treba da razmotrimo dve vrste dijagonala, one koje su paralelne sporednoj dijagonali (označimo ih sa $s$) i one koje su paralelne glavnoj dijagonali (označimo ih sa $g$). Dijagonale $s$ se mogu numerisati tako što se za svako polje $(r,c)$ dijagonala koja ga sadrži označi indeksom $r+c$. Dijagonale $g$ se mogu numerisati tako što se za svako polje $(r,c)$ dijagonala koja ga sadrži označi indeksom $r-c+n-1$ (na razliku se dodaje $n-1$ da bi indeksiranje bilo od nule). Na narednim dvema slikama prikazana je enumeracija dijagonala $s$ i $g$ na primeru šahovske table $8 \times 8$.

<img src="assets/queendiags.jpg" width="350px">

<img src="assets/queendiagg.jpg" width="350px">

Sada. ukoliko se dama dodaje na polje $(r, c)$, pored provere zauzetosti reda $r$, vrši se provera da li je zauzeta dijagonala $r+c$ iz grupe $s$ i dijagonala $r-c+n-1$ iz grupe $g$. Ukoliko je bar jedna dijagonala ili red zauzet, razmatra se drugo polje. Inače, dodaje se nova dama u rešenje, a odgovarajuće dijagonale i red se označavaju kao zauzeti.

#### Implementacija BnB algoritma za problem _n_ dama

In [8]:
import numpy as np

Funkcija `print_solution` štampa rešenje. Pozicije sa slovom Q označavaju pozicije na kojima je postavljana dama, a tačka prazno polje.

In [9]:
def print_solution(board):
    
    n = board.shape[0]
    
    for i in range(n):
        for j in range(n):
            print(board[i][j], end=' ')
        print()

Funkcija `is_free` proverava da li je polje opisano svojom vrstom i kolonom `(r, c)` slobodno. Nizovi `row_check`, `d1_check` i `d2_check` su nizovi sa logičkim vrednostima koji predstavljaju zauzeća vrsta i dijagonala.

In [10]:
def is_free(r, c, row_check, d1_check, d2_check):
    
    n = row_check.shape[0]
    if d1_check[r + c] or d2_check[r - c + n - 1] or row_check[r]:
        return False
    
    return True

Funkcija `bnb` predstavlja rekurzivnu implementacija BnB algoritma kojom se postavljaja dama na tablu `board` u kolonu `c`. Nizovi `row_check`, `d1_check` i `d2_check` su nizovi sa logičkim vrednostima koji predstavljaju zauzeća vrsta i dijagonala.

In [11]:
def bnb(board, c, row_check, d1_check, d2_check):
    
    n = board.shape[0]
    
    # ukoliko je postavljena dama u poslednjoj koloni, pronadjeno je resenje
    if c >= n:
        return True
    
    
    # r predstavlja indeks za redove
    for r in range(n):
  
        # prvo proveravamo da li je polje (r, c) slobodno
        if is_free(r, c, row_check, d1_check, d2_check):
            
            # ako jeste, postavljamo damu na polje (r, c)
            board[r][c] = 'Q'
            
            # i azuriramo nizove zauzeca za redove i dijagonale koji sadrze polje (r, c) 
            row_check[r] = True
            d1_check[r + c] = True
            d2_check[r - c + n - 1] = True
            
            # dalje pokusavamo sa postavljanjem dame u nerednu kolonu (c + 1)
            if bnb(board, c + 1, row_check, d1_check, d2_check):
                return True
            
            # u suprotnom prelazimo na bektreking deo: 
            
            # 1) dama se skida sa polja (r, c)
            board[r][c] = '.'          
            
            # 2) red i dijagonale koji se odnose na polje (r, c) se oznacavaju kao slobodne
            row_check[r] = False
            d1_check[r + c] = False
            d2_check[r - c + n - 1] = False
            
    return False

Funkcija `solve` inicijalizuje okruženje za problem table dimenzija `n x n` i ispisuje, ukoliko postoji, rešenje.

In [12]:
def solve(n):

    # pravimo tablu dimenzije n x n
    # slobodna polja su oznacena tackama, a zauzeta sa Q
    board = np.full((n, n), '.', dtype=str)
    
    # kreiramo niz koji cuva informacije o zauzetosti svakog reda
    row_check = np.full(n, False)
    
    # kreiramo niz koji cuva informacije o zauzetosti dijagonala iz grupe s
    d1_check = np.full(2*n-1, False)
    
    # kreiramo niz koji cuva informacije o zauzetosti dijagonala iz grupe g
    d2_check = np.full(2*n-1, False)
  
    # zatim pozivamo funkciju za rešavanje počevši od nulte kolone 
    
    # ukoliko nije pronadjeno nijedno resenje, ispisujemo poruku
    if not bnb(board, 0, row_check, d1_check, d2_check): 
        print('Nema resenja') 
        return False
    
    # inace, ispisujemo pronadjeno resenje
    print_solution(board)

In [13]:
solve(8)

Q . . . . . . . 
. . . . . . Q . 
. . . . Q . . . 
. . . . . . . Q 
. Q . . . . . . 
. . . Q . . . . 
. . . . . Q . . 
. . Q . . . . . 
